In [52]:
import configparser
from operator import le
import snowflake.connector
import pandas as pd

def getProp():
	global config
	config = configparser.RawConfigParser()
	config.read('../configuration.properties')
	return True

try:
    getProp()
    conn = snowflake.connector.connect(
        user=config['SNOWFLAKE']['Username'],
        password=config['SNOWFLAKE']['Password'],
        account=config['SNOWFLAKE']['Account'],
        warehouse=config['SNOWFLAKE']['Warehouse'],
        database=config['SNOWFLAKE']['Database'],
        schema=config['SNOWFLAKE']['Schema']
    )

    cur = conn.cursor()

    sql = "SELECT * FROM CFA_WEBSITE_DATA"
    cur.execute(sql)
    df = cur.fetch_pandas_all()

except Exception as e:
    print(e)

df.head()

255002: Optional dependency: 'pandas' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation


,topic_name,year,level,introduction,learning_outcome,summary,summary_page_link,pdf_file_link
0,Time-Series Analysis,2024.0,II,"As financial analysts, we often use time-seri...",The member should be able to: calculate and...,The predicted trend value of a time series in...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
1,Credit Analysis Models,2024.0,II,Credit analysis plays an important role in th...,The member should be able to: explain expec...,This reading has covered several important top...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
2,Introduction to Alternative Investments,2023.0,I,"In this section, we explain what alternative ...",The member should be able to: describe type...,This reading provides a comprehensive introduc...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
3,Credit Default Swaps,2024.0,II,Derivative instruments in which the underlyin...,The member should be able to: describe cred...,A credit default swap (CDS) is a contract betw...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
4,Valuation of Contingent Claims,2024.0,II,A contingent claim is a derivative instrument...,The member should be able to: describe and ...,This reading on the valuation of contingent cl...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...


In [40]:
# Filter the dataframe for the specified topic names
topic_names = [
"Overview of Equity Securities",
"Market Organization and Structure",
"Security Market Indexes"
]

filtered_df = df[df['topic_name'].isin(topic_names)]

# Remove the specified line from learning outcomes and expand into separate rows
expanded_list_cleaned = []
for index, row in filtered_df.iterrows():
    learning_outcomes = row['learning_outcome'].replace(
        "The member should be able to:", "").split(";")
    for outcome in learning_outcomes:
        cleaned_outcome = outcome.strip()
        if cleaned_outcome:  # Ensure the outcome is not empty
            expanded_list_cleaned.append({
                "topic_name": row['topic_name'],
                "learning_outcome": cleaned_outcome,
                "summary": row['summary']
            })

# Recreate the expanded dataframe without the unnecessary line
new_df = pd.DataFrame(expanded_list_cleaned)

# Append a blank "Technical Document" column
new_df["technical_document"] = ""

new_df.head()

,topic_name,learning_outcome,summary,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,This reading introduces how the financial syst...,
1,Market Organization and Structure,describe classifications of assets and markets,This reading introduces how the financial syst...,
2,Market Organization and Structure,"describe the major types of securities, curren...",This reading introduces how the financial syst...,
3,Market Organization and Structure,describe types of financial intermediaries and...,This reading introduces how the financial syst...,
4,Market Organization and Structure,compare positions an investor can take in an a...,This reading introduces how the financial syst...,


In [26]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
import os

def query_openai_for_technical_document(learning_outcome, summary, api_key):

    # completion llm
    llm = ChatOpenAI(
        openai_api_key=api_key,
        model_name='gpt-3.5-turbo',
        temperature=0.0,
        max_tokens=2048,
        frequency_penalty=0.1,
        presence_penalty=0.1,
    )

    messages = [
        SystemMessage(content="You are provided a SUMMARY below. Your goal is to provide a technical note that relates to the LEARNING OUTCOME. Include tables, figures and equations, ONLY if present in the CONTEXT. Ensure that you stickly stay within the CONTEXT and use no outside information. Answer in less than 300 words. No heading is needed. \LEARNING OUTCOMES:\n" + learning_outcome + "\n\nSUMMARY:\n" + summary + "\n\n"),
    ]

    response = llm.invoke(messages)
    response = response.content

    return response



In [44]:
getProp()
api_key = config['OPENAI']['OPENAI_API_KEY']

for index, row in new_df.iterrows():

    # Query OpenAI for each row and update the 'Technical Document' column
    new_df.at[index, 'technical_document'] = query_openai_for_technical_document(
        learning_outcome=row['learning_outcome'],
        summary=row['summary'],
        api_key=api_key
    )

    new_df.head()

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what 

In [30]:
new_df.head()

,topic_name,learning_outcome,summary,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,This reading introduces how the financial syst...,The main functions of the financial system inc...
1,Market Organization and Structure,describe classifications of assets and markets,This reading introduces how the financial syst...,The classification of assets and markets is cr...
2,Market Organization and Structure,"describe the major types of securities, curren...",This reading introduces how the financial syst...,"The major types of securities, currencies, con..."
3,Market Organization and Structure,describe types of financial intermediaries and...,This reading introduces how the financial syst...,Financial intermediaries play a crucial role i...
4,Market Organization and Structure,compare positions an investor can take in an a...,This reading introduces how the financial syst...,The financial system allows investors to take ...


In [53]:
new_df = new_df.drop(columns=['summary'])
new_df.head()

,topic_name,learning_outcome,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,The main functions of the financial system inc...
1,Market Organization and Structure,describe classifications of assets and markets,The classification of assets and markets is cr...
2,Market Organization and Structure,"describe the major types of securities, curren...","The major types of securities, currencies, con..."
3,Market Organization and Structure,describe types of financial intermediaries and...,Financial intermediaries play a crucial role i...
4,Market Organization and Structure,compare positions an investor can take in an a...,The financial system allows investors to take ...


In [54]:
new_df.to_csv('../data/output/technical_documents.csv', index=False)

In [51]:
# Grouping the data by topic_name and creating the markdown content
markdown_content = ""
for topic_name, group in new_df.groupby("topic_name"):
    markdown_content += f"## {topic_name}\n\n"
    for _, row in group.iterrows():
        markdown_content += f"# Learning Outcome:\n {row['learning_outcome']}\n"
        markdown_content += f"# Technical Document:\n {row['technical_document']}\n\n"

# Saving the markdown content to a .txt file
markdown_file_path = '../data/output/technical_documents_markdown.md'
with open(markdown_file_path, 'w') as file:
    file.write(markdown_content)

markdown_file_path

'../data/output/technical_documents_markdown.md'